In [1]:
# These are global imports
%pylab inline
import sys
import os
import numpy as np
import numpy.ma as ma
import pandas as pd
from scipy.spatial.distance import squareform
from scipy.stats import rankdata
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import ndimage
import cPickle as pickle
from joblib import Parallel,delayed
# Connect to an R session
import nibabel as nb


from mvpa2.suite import *
from scipy.io import loadmat
from scipy.spatial.distance import squareform,pdist
from scipy.stats import rankdata,pearsonr
from scipy import stats

#import prettyplotlib as ppl
#import brewer2mpl
from mpl_toolkits.mplot3d import Axes3D
from nibabel.spatialimages import ImageFileError

Populating the interactive namespace from numpy and matplotlib


/data/nielsond/python/envs/py27mvpa/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# Run ttest, make stat maps

In [2]:
run_name = 'perm_ham_stlim_SL4_zndist_1000perms_v1'
#define formula for glm, this is also a thing that may change run to run
fe_formula = 'val~ham+event'  # MODEL 3

#run_name='perm_ham_hamvivave_viv_ave_stlim_SL4_zndist_500perms'
#'perm_ham_hamremaverecoded_stlim_SL_zndist'
method = 'glm'
run_type = '%s_sl'%(method)

basedir = '/data/nielsond/expSamp/data/'
data_dir = os.path.join(basedir,'input')
data_file = 'rsa_dataset_gps_time_old_exclude_drop_viv_bin_scan_time_rem_ham.pickle'  
outdir = '/data/nielsond/expSamp/data/output/%s_sl/%s'%(method,run_name)
ss_out_pat = 'betas.%s.%s.%s.%03d.nii.gz'
combined_out_pat = '%s.%s.%s.nii.gz'
fmri_pat = 'input/stb.%s_stb_qwarp_e2a.nii.gz'
subj_mask_pat = 'input/%s_mask_GM_full_+tlrc.nii.gz'
stat_out_pat = '%s.%s.%s.%s.%03d.nii.gz'

mask_path='/data/nielsond/expSamp/data/input/MNI152_T1_2.5mm_GM_mask.nii.gz'
    

subjects = ['expSamp01',
'expSamp02',
'expSamp03',
'expSamp04',
'expSamp05',
'expSamp07',
'expSamp08',
'expSamp09',
'expSamp10']


#make output directory if it doesn't exist
if not os.path.exists(outdir):
    os.makedirs(outdir)
    
#load independent data
dat = np.load(os.path.join(data_dir,data_file))

In [3]:
outdir

'/data/nielsond/expSamp/data/output/glm_sl/perm_ham_stlim_SL4_zndist_1000perms_v1'

In [4]:
# Model 4
contrasts = {'event':1,'ham':2}
model_meas = {'deviance':(6),
             'scale':(7),
             'llf':(8),
             'count':(9)}


In [27]:
# Model 3
contrasts = {'correl':1,'event':2,'ham':3}
model_meas = {'deviance':(8),
             'scale':(9),
             'llf':(10),
             'count':(11)}


In [7]:
# Model 2
contrasts = {'event':1,'ham':2,'ham-viv_ave':3,'viv_ave':4}
model_meas = {'deviance':(10),
             'scale':(11),
             'llf':(12),
             'count':(13)}

In [ ]:
contrasts = {'correl':1,'event':2,'ham':3,'ham-viv_ave':4,
             'space':5, 'space:time':6,'time':7,'viv_ave':8}
model_meas = {'deviance':(18),
             'scale':(19),
             'llf':(20),
             'count':(21)}


In [87]:
contrasts = {'correl':1,'event':2,'ham':3,'ham-viv_ave':4,'viv_ave':5}
model_meas = {'deviance':(12),
             'scale':(13),
             'llf':(14),
             'count':(15)}

#1 event betas
#2 ham betas
#3 space betas
#4 space:time betas
#5 time betas
#6 Intercept tvals
#7 event tvals
#8 ham tvals
#9 space tvals
#10 space:time tvals
#11 time tvals
#12 deviance
#13 scale
#14 llf
#15 count


In [5]:
#grab dat mask
mask = nb.load(mask_path) 
#stdmask = nb.load(std_mask_path)
ind = (mask.get_data()>0) #& (stdmask.get_data()>0))
count_thresh = 20

In [6]:
#make output stats directory if it doesn't exist
stats_dir = os.path.join(outdir,'stats')
if not os.path.exists(stats_dir):
    os.makedirs(stats_dir)

  
def proc_perm(n):
    #betas = np.array([nb.load(os.path.join(outdir,ss_out_pat%(subj,run_name,run_type,n))).get_data()[ind] for subj in subjects]) #,n to ss_out_pat
    #betas = np.array([nb.load(os.path.join(outdir,ss_out_pat%(subj,run_name,run_type))).get_data()[ind] for subj in subjects]) #,n to ss_out_pat
    try:
        betas = np.array([nb.load(os.path.join(outdir,ss_out_pat%(subj,run_name,run_type,n))).get_data()[ind] for subj in subjects]) #,n to ss_out_pat
    except ImageFileError:
        return n
    c1=0 #checking any one factor is fine. If one is zero, the others are zero as well.
    zerosphere_ind=numpy.where(betas[:,:,c1] == 0)[1]  
    betas[:,zerosphere_ind,:]=np.NAN
    
    if len(betas) != 9:
        print "Perm %d not completed for all subjects"
    else:
        ttest_res = {}
        for c in contrasts.keys():
            if type(contrasts[c]) == int:
                ttest_res[c] = np.array(stats.ttest_1samp(betas[:,:,contrasts[c]],0.0,axis=0))
            else:
                ttest_res[c]=np.array(stats.ttest_rel(betas[:,:,contrasts[c][0]],
                                                 betas[:,:,contrasts[c][1]]))

        tvals = {fac:ttest_res[fac][0] for fac in ttest_res.keys() }
        
        #Adding min t stat map for the conjunction of keys in conj_keys
        #Work with min of abs t vals
        #tvals_conj = {fac:tvals[fac] for fac in conj_keys}
        #tval_mat = np.zeros((len(conj_keys),len(tvals_conj[conj_keys[0]])))
        #for idx in xrange(len(conj_keys)):
        #    tval_mat[idx]=tvals_conj.items()[idx][1]
        #tvals['conjunction']=np.absolute(tval_mat).min(axis=0)
        ##
        
        count = np.array(np.average(betas[:,:,model_meas['count']],axis=0))
        tvals_cmask = {fac:np.zeros(tvals[fac].shape) for fac in tvals.keys()}

        for fac in tvals.keys():
            tvals_cmask[fac][((~np.isnan(tvals[fac]))&(count>=count_thresh))] = tvals[fac][((~np.isnan(tvals[fac]))&(count>=count_thresh))]

        save_ind = np.nonzero(ind.flat>0)[0] #ind used here, which was the ind for mask
        measures = {'tvals_cmask':tvals_cmask}
        for m in measures.keys():
            for fac in tvals.keys():#ttest_res.keys():
                outfile = os.path.join(outdir,'stats',stat_out_pat%(fac,m,run_name,run_type,n)) #,n to the end
                #outfile = os.path.join(outdir,'vishstats',stat_out_pat%(fac,m,run_name,run_type)) #,n to the end
                dat_tosave = np.zeros(mask.get_data().shape) 
                dat_tosave.flat[save_ind] = measures[m][fac]
                img_out = nb.Nifti1Image(dat_tosave, 
                                 affine=mask.get_affine(), 
                                 header=mask.get_header())
                #print outfile
                img_out.to_filename(outfile)
                
        outfile = os.path.join(outdir,'stats',stat_out_pat%('count',m,run_name,run_type,n)) #,n to the end for perms
        #outfile = os.path.join(outdir,'vishstats',stat_out_pat%('count',m,run_name,run_type)) #,n to the end for perms
        dat_tosave = np.zeros(mask.get_data().shape)
        dat_tosave.flat[save_ind] = count
        img_out = nb.Nifti1Image(dat_tosave, 
                         affine=mask.get_affine(), 
                         header=mask.get_header())
        #print outfile
        img_out.to_filename(outfile)

In [ ]:
#Process permutations
fails = []
for n in np.arange(0,1000):
    fails.append(proc_perm(n))
fm = [f is not None for f in fails]
np.array(fails)[fm]

#Reprocesses failed permutation
new_fails = []
for n in [fails]:
    new_fails.append(proc_perm(n))

print(new_fails)

## TFCE

In [9]:
import subprocess as sp
from subprocess import CalledProcessError

In [10]:
m = "tvals_cmask"
pp_out_pat = '%s.%s.%s.%s.%s.%03d.nii.gz'
merge_in_pat = '%s.%s.%s.%s.%s.*.nii.gz'

merge_out_pat = '%s.%s.%s.%s.%s.nii.gz'

In [11]:
for fac in contrasts.keys():#ttest_res.keys():
    for n in range(1000):
        infile = os.path.join(outdir,'stats',stat_out_pat%(fac,m,run_name,run_type,n))
        tfce_pos_file = os.path.join(outdir,'stats',pp_out_pat%(fac,m,run_name,run_type,'tfce_pos',n))
        tfce_file = os.path.join(outdir,'stats',pp_out_pat%(fac,m,run_name,run_type,'tfce',n))
        sp.check_output(' '.join(["fslmaths",infile,"-thr 0", "-tfce 2 0.6666 26 ",tfce_pos_file]),shell = True)
        try:
            sp.check_output(' '.join(["fslmaths",infile,"-mul -1 -thr 0 -tfce 2 0.6666 26 -mul -1 -add",tfce_pos_file,tfce_file]),shell = True)
        except CalledProcessError as e:
            mint = float(sp.check_output(' '.join(['fslstats',infile,'-P 0']), shell = True).strip())
            if mint >= 0:
                sp.check_output(' '.join(["fslmaths",infile,"-thr 0", "-tfce 2 0.6666 26 ",tfce_file]),shell = True)
            else:
                raise e
        sp.check_output(' '.join(['rm',tfce_pos_file]), shell = True)

In [12]:
def get_thresh(f,q):
    perm_res = []
    with open(f,'r') as h:
        perm_res = h.readlines()
    perm_res = np.array([float(s.strip()) for s in perm_res])
    return np.percentile(perm_res,q)

In [13]:
thresh_d = {}
for fac in contrasts.keys():#ttest_res.keys(): 
    merge_input = os.path.join(outdir,'stats',merge_in_pat%(fac,m,run_name,run_type,'tfce'))
    merge_file = os.path.join(outdir,'stats',merge_out_pat%(fac,m,run_name,run_type,'merge'))
    max_file = os.path.join(outdir,'stats',merge_out_pat%(fac,m,run_name,run_type,'max_vals'))
    min_file = os.path.join(outdir,'stats',merge_out_pat%(fac,m,run_name,run_type,'min_vals'))

    sp.check_output(' '.join(["fslmerge -t",merge_file,merge_input]),shell = True)
    sp.check_output(' '.join(["fslstats -t",merge_file,"-P 100 >",max_file]), shell = True)
    sp.check_output(' '.join(["fslstats -t",merge_file,"-P 0 >",min_file]), shell = True)
    thresh_d[fac]={'lower':get_thresh(min_file,2.5),
                   'upper':get_thresh(max_file,97.5)}

In [14]:
thresh_d

{'event': {'lower': -32582.407373050002, 'upper': 27280.050488149998},
 'ham': {'lower': -31994.015869374998, 'upper': 28533.615820674993}}

In [18]:
cj = ('ham','ham-viv_ave')
fac = cj[0]+'_cj_'+cj[1]
for n in range(1000):
    cjin1 = os.path.join(outdir,'stats',stat_out_pat%(cj[0],m,run_name,run_type,n))
    cjin2 = os.path.join(outdir,'stats',stat_out_pat%(cj[1],m,run_name,run_type,n))
    cj1_post_file = os.path.join(outdir,'stats',pp_out_pat%(cj[0],m,run_name,run_type,'pos_t',n))
    cj2_negt_file = os.path.join(outdir,'stats',pp_out_pat%(cj[1],m,run_name,run_type,'neg_t',n))
    cj_t = os.path.join(outdir,'stats',pp_out_pat%(fac,m,run_name,run_type,'t',n))
    cj_tfce = os.path.join(outdir,'stats',pp_out_pat%(fac,m,run_name,run_type,'cj_tfce',n))

    sp.check_output(' '.join(['fslmaths',cjin1,'-thr 0',cj1_post_file]), shell = True)
    sp.check_output(' '.join(['fslmaths',cjin2,'-mul -1 -thr 0',cj2_negt_file]), shell = True)
    sp.check_output(' '.join(['fslmaths',cj1_post_file,'-min',cj2_negt_file,cj_t]), shell = True)
    sp.check_output(' '.join(['fslmaths',cj_t,'-thr 0 -tfce 2 0.6666 26',cj_tfce]), shell = True)

    #sp.check_output(' '.join(['rm', cj1_tfce_pos_file]), shell = True)
    #sp.check_output(' '.join(['rm', cj2_tfce_neg_file]), shell = True)
    


In [19]:
fac = cj[0]+'_cj_'+cj[1]
merge_input = os.path.join(outdir,'stats',merge_in_pat%(fac,m,run_name,run_type,'cj_tfce'))
merge_file = os.path.join(outdir,'stats',merge_out_pat%(fac,m,run_name,run_type,'tfce_merge'))
max_file = os.path.join(outdir,'stats',merge_out_pat%(fac,m,run_name,run_type,'tfce_max_vals'))
min_file = os.path.join(outdir,'stats',merge_out_pat%(fac,m,run_name,run_type,'tfce_min_vals'))

sp.check_output(' '.join(["fslmerge -t",merge_file,merge_input]),shell = True)
sp.check_output(' '.join(["fslstats -t",merge_file,"-a -P 100 >",max_file]), shell = True)
sp.check_output(' '.join(["fslstats -t",merge_file,"-a -P 0 >",min_file]), shell = True)
thresh_d[fac]={'upper_1sided':get_thresh(max_file,95)}

In [15]:
n = 0
for fac in contrasts.keys():
    tfce_file = os.path.join(outdir,'stats',pp_out_pat%(fac,m,run_name,run_type,'tfce',n))
    thresh_d[fac]['min'] = float(sp.check_output(' '.join(['fslstats',tfce_file,'-P 0']), shell = True).strip())
    thresh_d[fac]['max'] = float(sp.check_output(' '.join(['fslstats',tfce_file,'-P 100']), shell = True).strip())

#cjs = ('ham','ham-viv_ave')
#fac = cj[0]+'_cj_'+cj[1]
#cj_tfce = os.path.join(outdir,'stats',pp_out_pat%(fac,m,run_name,run_type,'cj_tfce',n))
#thresh_d[fac]['min'] = float(sp.check_output(' '.join(['fslstats',cj_tfce,'-P 0']), shell = True).strip())
#thresh_d[fac]['max'] = float(sp.check_output(' '.join(['fslstats',cj_tfce,'-P 100']), shell = True).strip())


In [18]:
thresh_out = os.path.join(outdir,'stats',"%s.tfce_thresholds.csv"%run_name)
pd.DataFrame(thresh_d).to_csv(thresh_out)

In [16]:
pd.DataFrame(thresh_d)

,event,ham
lower,-3.258241e+04,-31994.015869
max,1.238561e+06,44151.562500
min,4.705415e+03,-1935.239136
upper,2.728005e+04,28533.615821
